In [1]:
%%capture
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from statsmodels.tsa.arima_model import ARIMA, SARIMAX
import pmdarima as pm
import pickle

ImportError: cannot import name 'SARIMAX' from 'statsmodels.tsa.arima_model' (c:\users\atan0\github\fyp\fyp-code\sg_data\venv\lib\site-packages\statsmodels\tsa\arima_model.py)

In [ ]:
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return np.array(diff)
 
 # invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

In [ ]:
root_dir = "/content/drive/My Drive/Colab Notebooks/"

loc_2019_WEP = "2019_full_data.csv"
loc_2020_WEP= "2020_WEP_ONLY.csv"

model_loc = 'model_240_240_MLP_300_64.h5'
test_loc = '2020_WEP.csv'

wep_2020 = '2020_WEP_ONLY.csv'

In [ ]:
df_2019=pd.read_csv(loc_2019_WEP, index_col=0, parse_dates=True, usecols=['DATE','WEP ($/MWh)'])
df_2020=pd.read_csv(loc_2020_WEP, index_col=0, parse_dates=True)
df_2019.rename(columns={"WEP ($/MWh)": "WEP"})
df_2019_2020=pd.concat([df_2019.rename(columns={"WEP ($/MWh)": "WEP"}), df_2020])

In [ ]:
predict_date = "2020-01-25"
input_dataset_start = (datetime.strptime(predict_date, '%Y-%m-%d') - timedelta(days=21)).strftime('%Y-%m-%d')
input_dataset_end = (datetime.strptime(predict_date, '%Y-%m-%d') - timedelta(days=1)).strftime('%Y-%m-%d')

input_dataset = df_2019_2020['2019-12'].copy()
test_dataset = df_2019_2020['2020'].copy()
input_dataset

In [ ]:
model = pm.auto_arima(input_dataset.values, Seasonal=True, p=2 ,d=1, q=2, m=48)

print(model.summary())

In [ ]:
with open('arima_test.pkl', 'wb') as pkl:
    pickle.dump(model, pkl)

In [ ]:
with open('arima.pkl', 'rb') as pkl:
    pickle_preds = pickle.load(pkl)
"""
# seasonal difference
X = df_2020.values
weekly_timesteps = 336
differenced = difference(X, weekly_timesteps)
print (differenced.shape)
# fit model
model = ARIMA(differenced, order=(7,0,1))
model_fit = model.fit(disp=0)
# print summary of fit model
print(model_fit.summary())"""

In [ ]:
"""forecast = model_fit.forecast(steps=48)[0]
# invert the differenced forecast to something usable
history = [x for x in X]
predicted=[]
day = 1
for yhat in forecast:
  inverted = inverse_difference(history, yhat, weekly_timesteps)
  #print('Day %d: %f' % (day, inverted))
  history.append(inverted)
  predicted.append(inverted)
  day += 1

plt.plot(predicted)
df_2020['2020-01-01'].plot()
len(forecast)"""

pickle_preds.summary()